<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_RELATIVITY_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries (if running in Colab)
!pip install colab-env -q
!pip install mistralai -q

In [1]:
# Install necessary libraries (if running in Colab)
# !pip install colab-env-q
# !pip install mistralai -q

import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed (error handling)
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Relativity Proof Agent's response format
class RelativityProofResult(BaseModel):
    proof_aspect: str
    evidence_summary: str
    confidence_score: float
    key_experiments: list[str]
    relevant_theories: list[str]

print("Creating AI agents for Relativity Proof Domain...")

# 1. Scientific Literature Agent
literature_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for searching and retrieving scientific papers, historical documents, and experimental data related to physics and relativity.",
    name="scientific-literature-agent",
    tools=[
        {"type": "web_search"}, # For general web searches for scientific papers/articles
        {
            "type": "function",
            "function": {
                "name": "search_scientific_database",
                "description": "Search a scientific database for papers based on keywords, authors, or publication dates.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "keywords": {"type": "string", "description": "Keywords to search for (e.g., 'Michelson-Morley experiment', 'special relativity')."},
                        "author": {"type": "string", "description": "Optional: Author's name."},
                        "year_range": {"type": "string", "description": "Optional: Publication year range (e.g., '1900-1920')."}
                    },
                    "required": ["keywords"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "retrieve_document_content",
                "description": "Retrieve the full text or abstract of a scientific document given its ID or URL.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "document_id": {"type": "string", "description": "Identifier for the document (e.g., DOI, arXiv ID) or URL."},
                        "content_type": {"type": "string", "description": "Type of content to retrieve ('abstract' or 'full_text')."}
                    },
                    "required": ["document_id", "content_type"]
                }
            }
        }
    ]
)
print(f"Scientific Literature Agent '{literature_agent.name}' created with ID: {literature_agent.id}")

# 2. Theoretical Analysis Agent
theoretical_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for analyzing mathematical derivations, theoretical frameworks, and conceptual consistency of physics theories.",
    name="theoretical-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "verify_mathematical_derivation",
                "description": "Verify the correctness of a given mathematical derivation or equation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "equation_or_derivation": {"type": "string", "description": "The mathematical expression or derivation to verify (e.g., 'E=mc^2 derivation')."},
                        "context": {"type": "string", "description": "Context or assumptions for the derivation (e.g., 'special relativity postulates')."}
                    },
                    "required": ["equation_or_derivation", "context"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_theoretical_consistency",
                "description": "Analyze the consistency of a theoretical framework with known physical laws.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "theory_description": {"type": "string", "description": "Description of the theory to analyze (e.g., 'Lorentz transformations')."},
                        "known_laws": {"type": "string", "description": "Known physical laws for comparison (e.g., 'Maxwell's equations')."}
                    },
                    "required": ["theory_description", "known_laws"]
                }
            }
        }
    ]
)
print(f"Theoretical Analysis Agent '{theoretical_analysis_agent.name}' created with ID: {theoretical_analysis_agent.id}")

# 3. Experimental Simulation Agent
experimental_simulation_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for conceptually simulating classic physics experiments or thought experiments to test theoretical predictions.",
    name="experimental-simulation-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "simulate_light_propagation",
                "description": "Simulate the propagation of light under different reference frames.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "scenario_description": {"type": "string", "description": "Description of the scenario (e.g., 'light clock in moving train')."},
                        "parameters_json": {"type": "string", "description": "JSON string of simulation parameters (e.g., '{\"velocity\": 0.5c, \"duration\": 1s}')."}
                    },
                    "required": ["scenario_description", "parameters_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "model_gravitational_effects",
                "description": "Model gravitational effects predicted by general relativity.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "phenomenon": {"type": "string", "description": "Gravitational phenomenon to model (e.g., 'light bending around Sun', 'gravitational redshift')."},
                        "parameters_json": {"type": "string", "description": "JSON string of modeling parameters (e.g., '{\"mass\": 1.989e30, \"distance\": 1e11}')."}
                    },
                    "required": ["phenomenon", "parameters_json"]
                }
            }
        }
    ]
)
print(f"Experimental Simulation Agent '{experimental_simulation_agent.name}' created with ID: {experimental_simulation_agent.id}")

# 4. Proof Synthesis Agent
proof_synthesis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="proof-synthesis-agent",
    description="Agent for synthesizing evidence from literature, theoretical analysis, and simulations into a coherent proof or explanation of relativity.",
    instructions="Synthesize a comprehensive explanation or 'proof' of relativity based on provided evidence. Use the provided JSON schema for response format.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "relativity_proof_result",
                "schema": RelativityProofResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "synthesize_proof_components",
                "description": "Combine various pieces of evidence (experimental, theoretical, observational) into a structured argument for relativity.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "proof_components_json": {"type": "string", "description": "JSON string of different proof components (e.g., '{\"experimental\": [...], \"theoretical\": [...]}')."},
                        "target_theory": {"type": "string", "description": "The specific theory of relativity to prove ('Special Relativity' or 'General Relativity')."}
                    },
                    "required": ["proof_components_json", "target_theory"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "explain_conceptual_implications",
                "description": "Explain the conceptual implications of relativity's proof, linking it to fundamental physical principles.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "proof_summary": {"type": "string", "description": "A summary of the proof found."},
                        "key_concepts": {"type": "string", "description": "Key concepts to elaborate on (e.g., 'spacetime', 'mass-energy equivalence')."}
                    },
                    "required": ["proof_summary", "key_concepts"]
                }
            }
        }
    ]
)
print(f"Proof Synthesis Agent '{proof_synthesis_agent.name}' created with ID: {proof_synthesis_agent.id}")

# 5. Historical Context Agent (incorporating Newton, Galileo, Einstein, Hinton)
historical_context_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for providing historical context and the intellectual lineage of scientific discoveries, including the contributions of figures like Newton, Galileo, Einstein, and Hinton.",
    name="historical-context-agent",
    tools=[
        {"type": "web_search"}, # For searching historical scientific records
        {
            "type": "function",
            "function": {
                "name": "get_scientist_contributions",
                "description": "Retrieve key scientific contributions of a historical figure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "scientist_name": {"type": "string", "description": "Name of the scientist (e.g., 'Isaac Newton', 'Albert Einstein')."},
                        "field_of_study": {"type": "string", "description": "Optional: Specific field of study to narrow down contributions."}
                    },
                    "required": ["scientist_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_intellectual_synergy",
                "description": "Analyze the intellectual synergy or influence between different historical scientific figures.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "figure_A": {"type": "string", "description": "First scientific figure."},
                        "figure_B": {"type": "string", "description": "Second scientific figure."},
                        "common_theme": {"type": "string", "description": "Optional: Common theme or area of influence (e.g., 'relativity', 'mechanics', 'AI')."}
                    },
                    "required": ["figure_A", "figure_B"]
                }
            }
        }
    ]
)
print(f"Historical Context Agent '{historical_context_agent.name}' created with ID: {historical_context_agent.id}")


print("\nAll new Relativity Proof-related agents have been defined.")

# Mock functions for Relativity Proof domain tools
def search_scientific_database(keywords: str, author: str = None, year_range: str = None):
    """MOCK function to search a scientific database."""
    print(f"\n [DEBUG] MOCK CALL: search_scientific_database Keywords: '{keywords}', Author: '{author}', Year: '{year_range}'")
    if "Michelson-Morley" in keywords:
        return {"status": "success", "results": [{"title": "On the Relative Motion of the Earth and the Luminiferous Ether", "author": "A.A. Michelson, E.W. Morley", "year": "1887", "abstract": "Experiment showing no evidence for luminiferous ether."}, {"title": "The Aether and the Earth's Motion", "author": "H.A. Lorentz", "year": "1892", "abstract": "Discusses implications of Michelson-Morley."}]}
    if "E=mc^2" in keywords:
        return {"status": "success", "results": [{"title": "Does the Inertia of a Body Depend Upon Its Energy-Content?", "author": "A. Einstein", "year": "1905", "abstract": "Introduces mass-energy equivalence."}]}
    return {"status": "success", "results": [], "message": "No relevant papers found."}

def retrieve_document_content(document_id: str, content_type: str):
    """MOCK function to retrieve document content."""
    print(f"\n [DEBUG] MOCK CALL: retrieve_document_content Doc ID: '{document_id}', Type: '{content_type}'")
    if "Michelson-Morley" in document_id and content_type == "abstract":
        return {"status": "success", "content": "Abstract of Michelson-Morley: Demonstrated the constancy of the speed of light."}
    if "Einstein" in document_id and content_type == "full_text":
        return {"status": "success", "content": "Full text of Einstein's 1905 paper on mass-energy equivalence (excerpt): ...energy and mass are equivalent... E=mc^2..."}
    return {"status": "not_found", "message": "Document content not found."}

def verify_mathematical_derivation(equation_or_derivation: str, context: str):
    """MOCK function to verify mathematical derivation."""
    print(f"\n [DEBUG] MOCK CALL: verify_mathematical_derivation Derivation: '{equation_or_derivation}', Context: '{context}'")
    if "E=mc^2 derivation" in equation_or_derivation and "special relativity postulates" in context:
        return {"status": "correct", "details": "Derivation of E=mc^2 is consistent with special relativity principles."}
    if "Lorentz transformations" in equation_or_derivation:
        return {"status": "correct", "details": "Lorentz transformations are mathematically sound and form the basis of special relativity."}
    return {"status": "unverified", "details": "Could not verify derivation with provided context."}

def analyze_theoretical_consistency(theory_description: str, known_laws: str):
    """MOCK function to analyze theoretical consistency."""
    print(f"\n [DEBUG] MOCK CALL: analyze_theoretical_consistency Theory: '{theory_description}', Laws: '{known_laws}'")
    if "Lorentz transformations" in theory_description and "Maxwell's equations" in known_laws:
        return {"status": "consistent", "details": "Lorentz transformations ensure Maxwell's equations are invariant across inertial frames."}
    if "Newtonian mechanics" in theory_description and "speed of light constancy" in known_laws:
        return {"status": "inconsistent", "details": "Newtonian mechanics is inconsistent with the constancy of the speed of light, leading to the need for relativity."}
    return {"status": "analysis_incomplete", "details": "Further analysis needed for consistency."}

def simulate_light_propagation(scenario_description: str, parameters_json: str):
    """MOCK function to simulate light propagation."""
    print(f"\n [DEBUG] MOCK CALL: simulate_light_propagation Scenario: '{scenario_description}', Params: '{parameters_json}'")
    params = json.loads(parameters_json)
    if "light clock in moving train" in scenario_description and params.get("velocity"):
        return {"status": "simulation_complete", "result": "Time dilation observed: moving clock runs slower.", "visual_representation": "conceptual diagram of light path."}
    if "light bending around massive object" in scenario_description:
        return {"status": "simulation_complete", "result": "Light path bends in the presence of strong gravitational fields.", "visual_representation": "gravitational lensing diagram."}
    return {"status": "simulation_failed", "message": "Could not simulate scenario."}

def model_gravitational_effects(phenomenon: str, parameters_json: str):
    """MOCK function to model gravitational effects."""
    print(f"\n [DEBUG] MOCK CALL: model_gravitational_effects Phenomenon: '{phenomenon}', Params: '{parameters_json}'")
    params = json.loads(parameters_json)
    if "light bending around Sun" in phenomenon:
        return {"status": "modeling_complete", "prediction": "Deflection of light by ~1.75 arcseconds near Sun's limb.", "observed_match": "Matches Eddington's observations."}
    if "gravitational redshift" in phenomenon:
        return {"status": "modeling_complete", "prediction": "Light loses energy when climbing out of a gravitational well, shifting to redder wavelengths.", "observed_match": "Confirmed by Pound-Rebka experiment."}
    return {"status": "modeling_failed", "message": "Could not model phenomenon."}

def synthesize_proof_components(proof_components_json: str, target_theory: str):
    """MOCK function to synthesize proof components."""
    print(f"\n [DEBUG] MOCK CALL: synthesize_proof_components Components: '{proof_components_json}', Theory: '{target_theory}'")
    components = json.loads(proof_components_json)
    if target_theory == "Special Relativity" and "Michelson-Morley" in str(components) and "Lorentz transformations" in str(components):
        return {"status": "proof_synthesized", "proof_summary": "Proof of Special Relativity relies on the constancy of light speed (Michelson-Morley) and the mathematical framework of Lorentz transformations, leading to time dilation and length contraction.", "confidence": 0.95}
    if target_theory == "General Relativity" and "light bending" in str(components) and "gravitational redshift" in str(components):
        return {"status": "proof_synthesized", "proof_summary": "Proof of General Relativity is supported by observational evidence like light bending around massive objects and gravitational redshift, confirming spacetime curvature.", "confidence": 0.9}
    return {"status": "synthesis_incomplete", "proof_summary": "Not enough components to form a complete proof.", "confidence": 0.5}

def explain_conceptual_implications(proof_summary: str, key_concepts: str):
    """MOCK function to explain conceptual implications."""
    print(f"\n [DEBUG] MOCK CALL: explain_conceptual_implications Summary: '{proof_summary}', Concepts: '{key_concepts}'")
    if "time dilation" in key_concepts:
        return {"status": "explanation_generated", "explanation": "Time dilation means time passes differently for observers in relative motion, a direct consequence of the constancy of light speed.", "concepts_covered": ["time dilation", "relative motion"]}
    if "spacetime" in key_concepts:
        return {"status": "explanation_generated", "explanation": "Spacetime is a unified four-dimensional manifold where space and time are interwoven, and gravity is described as its curvature.", "concepts_covered": ["spacetime", "gravity", "curvature"]}
    return {"status": "explanation_incomplete", "explanation": "Could not elaborate on all concepts.", "concepts_covered": []}

def get_scientist_contributions(scientist_name: str, field_of_study: str = None):
    """MOCK function to retrieve scientist contributions."""
    print(f"\n [DEBUG] MOCK CALL: get_scientist_contributions Scientist: '{scientist_name}', Field: '{field_of_study}'")
    if "Isaac Newton" in scientist_name:
        return {"status": "success", "contributions": ["Laws of Motion", "Universal Gravitation", "Calculus"], "era": "Classical Mechanics"}
    if "Albert Einstein" in scientist_name:
        return {"status": "success", "contributions": ["Special Relativity", "General Relativity", "Photoelectric Effect"], "era": "Modern Physics"}
    if "Galileo Galilei" in scientist_name:
        return {"status": "success", "contributions": ["Principle of Relativity (Galilean)", "Telescopic Astronomy", "Kinematics"], "era": "Scientific Revolution"}
    if "Geoffrey Hinton" in scientist_name:
        return {"status": "success", "contributions": ["Backpropagation", "Deep Learning", "Neural Networks"], "era": "Artificial Intelligence"}
    return {"status": "not_found", "contributions": [], "message": "Scientist contributions not found."}

def analyze_intellectual_synergy(figure_A: str, figure_B: str, common_theme: str = None):
    """MOCK function to analyze intellectual synergy."""
    print(f"\n [DEBUG] MOCK CALL: analyze_intellectual_synergy Figure A: '{figure_A}', Figure B: '{figure_B}', Theme: '{common_theme}'")
    if "Galileo" in figure_A and "Einstein" in figure_B and "relativity" in common_theme:
        return {"status": "synergy_found", "details": "Einstein extended Galileo's principle of relativity from mechanics to electromagnetism and light, leading to special relativity.", "connection_type": "extension"}
    if "Newton" in figure_A and "Einstein" in figure_B and "gravity" in common_theme:
        return {"status": "synergy_found", "details": "Einstein's General Relativity superseded Newton's theory of gravity by describing gravity as spacetime curvature, but Newton's theory remains a valid approximation in weak fields.", "connection_type": "supersession_and_approximation"}
    if "Einstein" in figure_A and "Hinton" in figure_B and "AI" in common_theme:
        return {"status": "synergy_found", "details": "While not direct collaborators, both Einstein and Hinton revolutionized their fields by fundamentally rethinking established paradigms (physics and computation/learning, respectively). AI, influenced by Hinton's work, can now analyze complex data to potentially accelerate scientific discovery, a process Einstein exemplified.", "connection_type": "paradigm_shift_and_tooling"}
    return {"status": "no_direct_synergy", "details": "No direct intellectual synergy found for the given figures and theme."}

# Adding the missing internal_web_search_tool mock function
def internal_web_search_tool(*args, **kwargs):
    """Mock web search: General scientific information retrieved."""
    return "Mock web search: General scientific information retrieved."

# Master Tool Executor Mapping for Relativity Proof Domain
tool_executor = {
    "search_scientific_database": search_scientific_database,
    "retrieve_document_content": retrieve_document_content,
    "verify_mathematical_derivation": verify_mathematical_derivation,
    "analyze_theoretical_consistency": analyze_theoretical_consistency,
    "simulate_light_propagation": simulate_light_propagation,
    "model_gravitational_effects": model_gravitational_effects,
    "synthesize_proof_components": synthesize_proof_components,
    "explain_conceptual_implications": explain_conceptual_implications,
    "get_scientist_contributions": get_scientist_contributions,
    "analyze_intellectual_synergy": analyze_intellectual_synergy,
    "internal_web_search_tool": internal_web_search_tool, # Reusing the mock web search
}

# Function to standardize tools for client.chat.complete (remains the same)
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# Test Case Execution for Relativity Proof Agents
print("\n--- Executing Test Cases for Relativity Proof Agents (via chat completions) ---")
test_cases = [
    {
        "agent": literature_agent,
        "name": "Scientific Literature Agent",
        "query": "Find papers on the Michelson-Morley experiment.",
        "expected_tool_call": "search_scientific_database"
    },
    {
        "agent": literature_agent,
        "name": "Scientific Literature Agent",
        "query": "Retrieve the abstract of Einstein's 1905 paper on E=mc^2.",
        "expected_tool_call": "retrieve_document_content"
    },
    {
        "agent": theoretical_analysis_agent,
        "name": "Theoretical Analysis Agent",
        "query": "Verify the mathematical derivation of E=mc^2 based on special relativity postulates.",
        "expected_tool_call": "verify_mathematical_derivation"
    },
    {
        "agent": theoretical_analysis_agent,
        "name": "Theoretical Analysis Agent",
        "query": "Analyze the consistency of Lorentz transformations with Maxwell's equations.",
        "expected_tool_call": "analyze_theoretical_consistency"
    },
    {
        "agent": experimental_simulation_agent,
        "name": "Experimental Simulation Agent",
        "query": "Simulate a light clock in a train moving at 0.5c to show time dilation.",
        "expected_tool_call": "simulate_light_propagation"
    },
    {
        "agent": experimental_simulation_agent,
        "name": "Experimental Simulation Agent",
        "query": "Model the bending of light around the Sun due to gravity.",
        "expected_tool_call": "model_gravitational_effects"
    },
    {
        "agent": proof_synthesis_agent,
        "name": "Proof Synthesis Agent",
        "query": "Synthesize a proof for Special Relativity using evidence from Michelson-Morley and Lorentz transformations.",
        "expected_tool_call": "synthesize_proof_components"
    },
    {
        "agent": proof_synthesis_agent,
        "name": "Proof Synthesis Agent",
        "query": "Explain the conceptual implications of spacetime from the proof of General Relativity.",
        "expected_tool_call": "explain_conceptual_implications"
    },
    {
        "agent": historical_context_agent,
        "name": "Historical Context Agent",
        "query": "What were Isaac Newton's key contributions to physics?",
        "expected_tool_call": "get_scientist_contributions"
    },
    {
        "agent": historical_context_agent,
        "name": "Historical Context Agent",
        "query": "Analyze the intellectual synergy between Galileo and Einstein regarding the principle of relativity.",
        "expected_tool_call": "analyze_intellectual_synergy"
    },
    {
        "agent": historical_context_agent,
        "name": "Historical Context Agent",
        "query": "How did Einstein's work relate to Newton's theory of gravity?",
        "expected_tool_call": "analyze_intellectual_synergy"
    },
    {
        "agent": historical_context_agent,
        "name": "Historical Context Agent",
        "query": "Explore the synergy between Einstein and Hinton in the context of AI and scientific discovery.",
        "expected_tool_call": "analyze_intellectual_synergy"
    },
    {
        "agent": literature_agent,
        "name": "Scientific Literature Agent (Web Search Example)",
        "query": "What are the latest theories on quantum gravity?",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"]

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")

    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)
        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )
        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1)

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output_content}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list,
            )
            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message)
        else:
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")

print("\n--- All test cases execution complete. --")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Creating AI agents for Relativity Proof Domain...
Scientific Literature Agent 'scientific-literature-agent' created with ID: ag_0197c27266c376f096a7c02f6f5cbb5c
Theoretical Analysis Agent 'theoretical-analysis-agent' created with ID: ag_0197c2726874739c8d50ec6f285d6dbc
Experimental Simulation Agent 'experimental-simulation-agent' created with ID: ag_0197c27269377670a834aea104d97871
Proof Synthesis Agent 'proof-synthesis-agent' created with ID: ag_0197c27269f6715d90ea8962f30e398d
Historical Context Agent 'historical-context-agent' created with ID: ag_0197c2726aab737388ee667fc7774aaf

All new Relativity Proof-related agents have been defined.

--- Executing Test Cases for Relativity Proof Agents (via chat completions) ---

--- Executing Test Case for the Scientific Literature Agent ---
User: Find papers on the Michelson-Morley experiment.

Scientific Literatu